In [1]:
%reload_ext autoreload

In [36]:
# Imports

%load_ext autoreload
%autoreload 2

%matplotlib inline
import numpy as np
import random
import csv
import pandas as pd

from fastai.imports import *
from fastai.structured import *
from matplotlib import pyplot
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.metrics import f1_score,\
    accuracy_score, confusion_matrix,\
    precision_score, recall_score,\
    roc_curve, roc_auc_score,\
    cohen_kappa_score, mean_absolute_error,\
    precision_recall_curve, auc,\
    average_precision_score

set_plot_sizes(12,14,16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
!ls ../lib/

__pycache__ utils.py


In [38]:
import csv

sys.path.append('../lib')
import utils

In [39]:
# Load in data

PATH = "../data/"
df_raw = pd.read_csv(f'{PATH}churn.csv', low_memory=False, 
                     parse_dates=['canceldate', 'licence_registration_date', 'golive_date'])

In [40]:
print(len(df_raw[df_raw.churned == 0]), df_raw[df_raw.churned == 0].shape)
df_processed = utils.prepare_data(df_raw)
print(len(df_processed), df_processed.shape)
df_filtered = df_processed[df_processed.licence_registration_Year >2017]
df_data, y_data, nas = proc_df(df_filtered, 'churned')
print(len(df_data), df_data.shape)

2780 (2780, 38)
5889 (5889, 99)
1351 (1351, 102)


/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [41]:
joblib_file = "churn_model.pkl"

# Load from file
churn_model = joblib.load(joblib_file)

predictions = churn_model.predict(df_data)
probability = churn_model.predict_proba(df_data)

now = datetime.datetime.now()
array = []
for i in range(len(df_filtered[df_filtered.churned == 0].username)):
    if predictions[i] == 1:
        array.append([now,df_filtered[df_filtered.churned == 0].username.iloc[i],df_filtered[df_filtered.churned == 0].url.iloc[i],df_filtered[df_filtered.churned == 0].annual_revenue.iloc[i],probability[i][1],predictions[i]])
        

print(len(df_raw[df_raw.churned == 0]), len(array))

2780 243


In [42]:
count = 0

churn_concerns = []

for i in range(len(array)):
    if array[i][4] > .7:
        count += 1
        churn_concerns.append(array[i])

In [43]:
print(count)

80


In [44]:
now = datetime.datetime.now()

sorted_array = sorted(churn_concerns, key=lambda x: x[3], reverse=True)

#for i in range(len(sorted_array)):
for i in range(30):
    print(f"{sorted_array[i][0]},{sorted_array[i][1]},{sorted_array[i][2]},{sorted_array[i][3]},{sorted_array[i][4]},{sorted_array[i][5]}")
    with open('../data/predictions.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows(sorted_array)

writeFile.close()

2019-10-05 15:33:26.859530,N062160,www.optimusenergy.com.au,5822790.28,0.7101540482752969,1
2019-10-05 15:33:26.859530,N052273,logitechshop.neto.com.au,3161843.85,0.7149377498979065,1
2019-10-05 15:33:26.859530,N050357,www.australianhomebrewing.com.au,1913375.39,0.7204407690954798,1
2019-10-05 15:33:26.859530,N056451,www.bosscoauto.com.au,1296910.26,0.7212239210197177,1
2019-10-05 15:33:26.859530,N055220,www.xdw.com.au,800290.23,0.7180261193111497,1
2019-10-05 15:33:26.859530,N054211,www.funeralproducts.com.au,649310.58,0.71853871584007,1
2019-10-05 15:33:26.859530,N054344,www.adolfodominguez.com.au,585114.98,0.7228967016882764,1
2019-10-05 15:33:26.859530,N052113,www.willowspoolshop.com.au,485805.76,0.7096051287422293,1
2019-10-05 15:33:26.859530,N054407,www.gssautoparts.com.au,394472.03,0.7241069100373339,1
2019-10-05 15:33:26.859530,N056267,wheel-house-bik.neto.com.au,393561.83,0.7204315937996917,1
2019-10-05 15:33:26.859530,N059921,www.directcarparts.com.au,330861.34,0.715224143400